In [11]:
import numpy as np
import pandas as pd

In [12]:
years = [2019,2020,2021,2022,2023]
conferences = ['ACC', 'Big10', 'Big12', 'SEC', 'PAC12']

for year in years:
    for con in conferences:

        #Player Stats by Season
        p_stats_season = pd.read_csv(f'Player Stats/Player Stats By Season {con} {year}.csv')
        #Remove 'Team' Player
        p_stats_season = p_stats_season[p_stats_season['Player'] != ' Team']

        #Group Player Stats by PlayerId
        p_stats_season = p_stats_season[['PlayerId', 'StatType', 'Stat']].groupby(['PlayerId', 'StatType']).sum().unstack()
        p_stats_season.columns = p_stats_season.columns.droplevel(0)
        p_stats_season.reset_index(inplace = True)
        p_stats_season = p_stats_season.rename_axis(None, axis=1)
        p_stats_season.index = p_stats_season.index.map(int)

        #Player usage data
        p_usage = pd.read_csv(f'Player Usage/Player Usage {con} {year}.csv')
        p_usage = p_usage.rename(columns = {'Name':'Player', 'Id': 'PlayerId'}).sort_values(by = 'Player').reset_index(drop = True)

        #Merge Player Stats and Player Usage
        merged = p_usage.merge(p_stats_season, on = 'PlayerId', how = 'outer')
        merged['Season'] = merged['Season'].apply(lambda x: year)
        merged['Conference'] = merged['Conference'].apply(lambda x: con)


        #2023 ALL CFB Player Basic Info
        p_info = pd.read_csv(f'Player Basic Info/{year} Players Basic Info.csv')
        p_info['Player'] = p_info['First Name'] + ' '+ p_info['Last Name']
        p_info.rename(columns = {'Id': 'PlayerId'}, inplace=True)
        p_info.drop(columns = ['First Name', 'Last Name'], inplace = True)
        p_info.sort_values(by = 'Player', inplace = True)
        p_info.dropna(inplace=True)

        #Team Records
        t_records = pd.read_csv(f'Team Records/Team Records {con} {year}.csv')
        t_records.drop(columns=['Year', 'Conference'], inplace = True)

        #Team Basic Stats
        t_b_stats = pd.read_csv(f'Team Stats/Team Basic Stats {con} {year}.csv')
        t_b_stats = t_b_stats[['Team', 'StatName', 'StatValue']].groupby(['Team', 'StatName']).sum().unstack()
        t_b_stats.columns = t_b_stats.columns.droplevel(0)
        t_b_stats.reset_index(inplace = True)
        t_b_stats = t_b_stats.rename_axis(None, axis=1)
        t_b_stats = t_b_stats.add_prefix('Team ')
        t_b_stats.rename(columns={'Team Team': 'Team'},  inplace=True)

        #Merge to fill in missing player info
        merged2 = merged.merge(p_info, on = 'PlayerId', how = 'left', suffixes=[None, '_y'])
        merged2['Position'] = merged2['Position'].combine_first(merged2['Position_y'])
        merged2['Team'] = merged2['Team'].combine_first(merged2['Team_y'])
        merged2['Player'] = merged2['Player'].combine_first(merged2['Player_y'])
        merged2.drop(columns=['Team_y', 'Position_y', 'Player_y'], inplace= True)
        merged2.sort_values(by = 'PlayerId')
        merged2.dropna(subset = ['Player'], inplace=True)

        #Merge Team Record Info
        merged3 = merged2.merge(t_records, on = 'Team')

        #Merge team basic stats
        merged4 = merged3.merge(t_b_stats, on = 'Team')

        #Load transfer data 
        transfer = pd.read_csv(f'Transfer Data/{year+1} Transfer.csv')
        transfer['Player'] = transfer['FirstName'] + ' ' + transfer['LastName']
        transfer.rename(columns = {'Origin': 'Team'}, inplace = True)
        transfer = transfer[['Player','Team', 'Position', 'Stars']]
        transfer = transfer.assign(Transfer_Portal ='Yes')

        #Merge Data with Transfer Data 
        final_merged = merged4.merge(transfer, on = ['Player', 'Team', 'Position'], how = 'left')
        final_merged['Transfer_Portal'] = final_merged['Transfer_Portal'].fillna('No')

        final_merged.to_csv(f'Merged Datasets Con Year/{con} {year}.csv')

In [13]:
for con in conferences:
    #take in each year
    set0 = pd.read_csv(f'Merged Datasets Con Year/{con} 2019.csv')
    set1 = pd.read_csv(f'Merged Datasets Con Year/{con} 2020.csv')
    set2 = pd.read_csv(f'Merged Datasets Con Year/{con} 2021.csv')
    set3 = pd.read_csv(f'Merged Datasets Con Year/{con} 2022.csv')
    set4 = pd.read_csv(f'Merged Datasets Con Year/{con} 2023.csv')

    #merge each year
    merged = pd.concat([set0,set1, set2, set3, set4])

    #export to new single conference dataset
    merged.to_csv(f'Merged Datasets Con/{con}.csv')

In [14]:
acc = pd.read_csv(f'Merged Datasets Con/ACC.csv')
big10 = pd.read_csv(f'Merged Datasets Con/Big10.csv')
big12 = pd.read_csv(f'Merged Datasets Con/Big12.csv')
sec = pd.read_csv(f'Merged Datasets Con/SEC.csv')
pac12 = pd.read_csv(f'Merged Datasets Con/PAC12.csv')

final_concat = pd.concat([acc, big10, big12, sec, pac12])

final_concat = final_concat.drop(columns=['Unnamed: 0.1','Unnamed: 0'])

#There was a mishap somewhere in the data downloading, and I need to define the columns we're going to include
cols = ['Season', 'PlayerId', 'Player', 'Position', 'Team', 'Conference',
       'Usage Overall', 'Usage Pass', 'Usage Rush', 'Usage FirstDown',
       'Usage SecondDown', 'Usage ThirdDown', 'Usage StandardDowns',
       'Usage PassingDowns', 'ATT', 'AVG', 'CAR', 'COMPLETIONS', 'FGA', 'FGM',
       'FUM', 'INT', 'In 20', 'LONG', 'LOST', 'NO', 'PCT', 'PD', 'PTS',
       'QB HUR', 'REC', 'SACKS', 'SOLO', 'TB', 'TD', 'TFL', 'TOT', 'XPA',
       'XPM', 'YDS', 'YPA', 'YPC', 'YPP', 'YPR', 'Division', 'ExpectedWins',
       'Total Games', 'Total Wins', 'Total Losses', 'Total Ties',
       'ConferenceGames Games', 'ConferenceGames Wins',
       'ConferenceGames Losses', 'ConferenceGames Ties', 'HomeGames Games',
       'HomeGames Wins', 'HomeGames Losses', 'HomeGames Ties',
       'AwayGames Games', 'AwayGames Wins', 'AwayGames Losses',
       'AwayGames Ties', 'Team firstDowns', 'Team fourthDownConversions',
       'Team fourthDowns', 'Team fumblesLost', 'Team fumblesRecovered',
       'Team games', 'Team interceptionTDs', 'Team interceptionYards',
       'Team interceptions', 'Team kickReturnTDs', 'Team kickReturnYards',
       'Team kickReturns', 'Team netPassingYards', 'Team passAttempts',
       'Team passCompletions', 'Team passesIntercepted', 'Team passingTDs',
       'Team penalties', 'Team penaltyYards', 'Team possessionTime',
       'Team puntReturnTDs', 'Team puntReturnYards', 'Team puntReturns',
       'Team rushingAttempts', 'Team rushingTDs', 'Team rushingYards',
       'Team sacks', 'Team tacklesForLoss', 'Team thirdDownConversions',
       'Team thirdDowns', 'Team totalYards', 'Team turnovers', 'Stars',
       'Transfer_Portal']

final_concat = final_concat[cols]

position_dict = {}
team_dict = {}
con_dict = {}

for i,pos in enumerate(final_concat.Position.unique()):
    position_dict[pos] = i

for i,team in enumerate(final_concat.Team.unique()):
    team_dict[team] = i

for i,con in enumerate(final_concat.Conference.unique()):
    con_dict[con] = i

final_concat['PositionId'] = final_concat['Position'].map(position_dict)
final_concat['TeamId'] = final_concat['Team'].map(team_dict)
final_concat['ConferenceId'] = final_concat['Conference'].map(con_dict)

final_concat.to_csv('Final Dataset.csv', index=False)

In [15]:
data = pd.read_csv("Final Dataset.csv")

data

/var/folders/5r/dp58bhsd7wvbt6d73cxcyj840000gn/T/ipykernel_61070/3644852142.py:1: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Final Dataset.csv")


,Season,PlayerId,Player,Position,Team,Conference,Usage Overall,Usage Pass,Usage Rush,Usage FirstDown,...,Team tacklesForLoss,Team thirdDownConversions,Team thirdDowns,Team totalYards,Team turnovers,Stars,Transfer_Portal,PositionId,TeamId,ConferenceId
0,2019,4240710,A.J. Davis,RB,Pittsburgh,ACC,0.1587,0.0525,0.3056,0.1700,...,124,84,211,4946,22,NaN,No,0,0,0
1,2019,4036066,Aaron Mathews,WR,Pittsburgh,ACC,0.0228,0.0394,0.0024,0.0246,...,124,84,211,4946,22,NaN,No,1,0,0
2,2019,4240724,Dontavius Butler-Jenkins,WR,Pittsburgh,ACC,0.0083,0.0150,0.0000,0.0025,...,124,84,211,4946,22,NaN,No,1,0,0
3,2019,4430681,Jared Wayne,WR,Pittsburgh,ACC,0.0207,0.0356,0.0024,0.0320,...,124,84,211,4946,22,NaN,No,1,0,0
4,2019,4373334,John Vardzel,WR,Pittsburgh,ACC,0.0041,0.0075,0.0000,0.0049,...,124,84,211,4946,22,NaN,No,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16796,2023,5077303,Jacob Lane,DE,Washington,PAC12,NaN,NaN,NaN,NaN,...,63,76,174,6931,20,NaN,No,14,57,4
16797,2023,5078420,Leroy Bryant,CB,Washington,PAC12,NaN,NaN,NaN,NaN,...,63,76,174,6931,20,NaN,No,8,57,4
16798,2023,5082671,Tristan Warner,S,Washington,PAC12,NaN,NaN,NaN,NaN,...,63,76,174,6931,20,NaN,No,9,57,4
16799,2023,5082675,Griffin Miller,LB,Washington,PAC12,NaN,NaN,NaN,NaN,...,63,76,174,6931,20,NaN,No,4,57,4
